# Data exploration of pipeline accidents
## Chris Stafford
### 3/2/18

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import folium
import folium.plugins as plugins
import os
from datetime import datetime as dt
from dateutil import parser
%matplotlib inline

Read data

In [3]:
df = pd.read_csv('data/pipeline-accidents.csv')


Show data scheme and get a sence for what information the data contains

In [4]:
df.describe()

,Report Number,Supplemental Number,Accident Year,Operator ID,Accident Latitude,Accident Longitude,Unintentional Release (Barrels),Intentional Release (Barrels),Liquid Recovery (Barrels),Net Loss (Barrels),...,Other Fatalities,Public Fatalities,All Fatalities,Property Damage Costs,Lost Commodity Costs,Public/Private Property Damage Costs,Emergency Response Costs,Environmental Remediation Costs,Other Costs,All Costs
count,2.795000e+03,2795.000000,2795.000000,2795.000000,2795.000000,2795.000000,2795.000000,1209.000000,2795.000000,2795.000000,...,8.000000,8.000000,8.00000,2.788000e+03,2.791000e+03,2.785000e+03,2.789000e+03,2.787000e+03,2.779000e+03,2.795000e+03
mean,2.013293e+07,19433.319499,2013.187835,21388.476565,35.781608,-95.682691,207.245843,238.533623,75.051792,132.194050,...,0.125000,0.625000,1.25000,1.122963e+05,9.805299e+03,2.512195e+04,2.918911e+05,3.628094e+05,3.435604e+04,8.340332e+05
std,1.982098e+04,1724.815370,1.972102,12430.973428,5.652599,12.328430,1368.667813,2781.771453,546.188247,1185.019252,...,0.353553,0.744024,0.46291,8.709947e+05,6.384075e+04,5.243581e+05,4.617076e+06,1.206887e+07,6.191235e+05,1.657830e+07
min,2.010000e+07,15072.000000,2010.000000,300.000000,18.448010,-158.099930,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.00000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.012004e+07,17978.500000,2012.000000,11169.000000,30.909375,-100.587636,0.480000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.00000,1.000000e+02,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.039500e+03
50%,2.013037e+07,19502.000000,2013.000000,26041.000000,34.925440,-95.488870,2.000000,0.000000,0.740000,0.000000,...,0.000000,0.500000,1.00000,3.000000e+03,1.000000e+02,0.000000e+00,2.520000e+03,8.000000e+02,0.000000e+00,2.312900e+04
75%,2.015023e+07,20996.500000,2015.000000,31618.000000,40.264062,-91.089463,20.000000,0.000000,8.000000,2.000000,...,0.000000,1.000000,1.25000,2.500000e+04,7.000000e+02,0.000000e+00,1.550000e+04,1.500000e+04,0.000000e+00,1.172325e+05
max,2.017003e+07,22049.000000,2017.000000,99043.000000,70.261265,104.263399,30565.000000,70191.000000,18245.000000,30565.000000,...,1.000000,2.000000,2.00000,2.700000e+07,1.417839e+06,2.300000e+07,1.770000e+08,6.350000e+08,2.235000e+07,8.405261e+08


In [5]:
df.head()


,Report Number,Supplemental Number,Accident Year,Accident Date/Time,Operator ID,Operator Name,Pipeline/Facility Name,Pipeline Location,Pipeline Type,Liquid Type,...,Other Fatalities,Public Fatalities,All Fatalities,Property Damage Costs,Lost Commodity Costs,Public/Private Property Damage Costs,Emergency Response Costs,Environmental Remediation Costs,Other Costs,All Costs
0,20100016,17305,2010,1/1/2010 7:15 AM,32109,ONEOK NGL PIPELINE LP,KINDER MORGAN JCT,ONSHORE,ABOVEGROUND,"HVL OR OTHER FLAMMABLE OR TOXIC FLUID, GAS",...,NaN,NaN,NaN,110.0,1517.0,0.0,0.0,0.0,0.0,1627
1,20100254,17331,2010,1/4/2010 8:30 AM,15786,PORTLAND PIPELINE CORP,24-INCH MAIN LINE,ONSHORE,ABOVEGROUND,CRUDE OIL,...,NaN,NaN,NaN,4000.0,8.0,0.0,0.0,0.0,0.0,4008
2,20100038,17747,2010,1/5/2010 10:30 AM,20160,"PETROLOGISTICS OLEFINS, LLC",NaN,ONSHORE,ABOVEGROUND,"HVL OR OTHER FLAMMABLE OR TOXIC FLUID, GAS",...,NaN,NaN,NaN,0.0,200.0,0.0,0.0,0.0,0.0,200
3,20100260,18574,2010,1/6/2010 7:30 PM,11169,"ENBRIDGE ENERGY, LIMITED PARTNERSHIP",SUPERIOR TERMINAL,ONSHORE,UNDERGROUND,CRUDE OIL,...,NaN,NaN,NaN,200.0,40.0,0.0,11300.0,0.0,0.0,11540
4,20100030,16276,2010,1/7/2010 1:00 PM,300,"PLAINS PIPELINE, L.P.",RED RIVER EAST,ONSHORE,UNDERGROUND,CRUDE OIL,...,NaN,NaN,NaN,20000.0,150.0,0.0,7500.0,2000.0,0.0,29650


Select Location Data

In [6]:
loc_df = df.filter(['Report Number','Supplemental Number', 'Operator Name', 'Accident Latitude', 'Accident Longitude','Pipeline Type','Liquid Explosion'])

In [7]:
loc_df.describe()

,Report Number,Supplemental Number,Accident Latitude,Accident Longitude
count,2.795000e+03,2795.000000,2795.000000,2795.000000
mean,2.013293e+07,19433.319499,35.781608,-95.682691
std,1.982098e+04,1724.815370,5.652599,12.328430
min,2.010000e+07,15072.000000,18.448010,-158.099930
25%,2.012004e+07,17978.500000,30.909375,-100.587636
50%,2.013037e+07,19502.000000,34.925440,-95.488870
75%,2.015023e+07,20996.500000,40.264062,-91.089463
max,2.017003e+07,22049.000000,70.261265,104.263399


Drop null and Nan values in location data

In [8]:
loc_df.dropna(axis = 0,how = 'any',inplace = True)

## Where are accident hotspots?

### Create an interactive heat map for accident incidence over the full time frame of the available data

In [10]:
lat_long = loc_df.filter(['Accident Latitude', 'Accident Longitude'])
lat_long = lat_long.values.tolist()

In [21]:
#Create map
m = folium.Map([35., -90.], tiles='stamentoner', zoom_start=5)

plugins.HeatMap(lat_long).add_to(m)

m

## How does location of accident incidence change over time?

turn date column into useful date-time classifications

In [ ]:
df['date'] = df['Accident Date/Time']
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['year'] = df['date'].dt.year
df['weekday'] = df['date'].dt.weekday_name
df['day_of_week'] = df['date'].dt.dayofweek

In [ ]:
loc_time_df = df.filter(['Report Number','Accident Latitude', 'Accident Longitude','month','year'])

Split location data by year, month of incident.
Create interactive heatmap animation of accident incident by month.

In [76]:
#Generate Data for folium map

year = loc_time_df.year.unique()
month = loc_time_df.month.unique()

data = []
for y in year:
    for mo in month:
        locmy_df = loc_time_df[(loc_time_df['month'] == mo) & (loc_time_df['year'] == y)]
        lat_long = locmy_df.filter(['Accident Latitude', 'Accident Longitude']).values.tolist()
        data.append(lat_long)

In [78]:
#Create Map
m = folium.Map([38., -90.], tiles='stamentoner', zoom_start=4)

hm = plugins.HeatMapWithTime(data)

hm.add_to(m)

m